In [1]:
import numpy as np
import sympy

In [2]:
perch_length = np.array(
    [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 
     36.5, 36.0, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
     )
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [3]:
class Activation:
  """
  데이터의 활성화 함수 연산 결과를 얻기 위함
  추가로 역전파 계산에 필요한 활성화 함수의 미분 함수
  """
  # 활성화 함수
  fx = 0
  # 활성화 함수의 미분 함수
  d_fx = 0

  def sigmoid_fun(self, input):
    """
    시그모이드 함수
    자연 상수의 지수 함수를 사용한 0~1 사이의 값 출력
    Args:
      input : 입력값
    Return:
      0~1 사이의 값
    """
    x = sympy.Symbol('x')
    result = []

    self.fx = 1/(1+np.e**(-x))

    self.d_fx = sympy.diff(self.fx, x)

    for i in range(input.shape[0]):
      result.append(self.fx.subs([(x, input[i][0])]))

    return np.array(result).reshape(-1, 1)

  def d_sigmoid_fun(self, input):
    """
    시그모이드 함수의 미분값, 노드의 출력이 입력된다.
    arg
      input : 입력값
    return
      시그모이드 미분 함수의 계산 값
    """
    result = []

    x = sympy.Symbol('x')

    for i in range(input.shape[0]):
      result.append(self.d_fx.subs([(x,input[i][0])]))

    return np.array(result).reshape(-1, 1)

In [4]:
class LSTM:
  """
  LSTM 의 계산
  """
  # i 를 생성하기 위한 가중치 
  w_xi = []
  w_hi = []
  b_i = []

  # g 를 생성하기 위한 가중치
  w_xg = []
  w_hg = []
  b_g = []

  # f 를 생성하기 위한 가중치
  w_xf = []
  w_hf = []
  b_f = []

  # o 를 생성하기 위한 가중치
  w_xo = []
  w_ho = []
  b_o = []

  def input_gate_i(self, input_x, input_h, output):
    w_xi = np.random.rand(output.shape[0], input_x.shape[0])
    w_hi = np.random.rand(output.shape[0], output.shape[1])
    b_i = np.random.rand(output.shape[0],1)

    self.w_xi.append(w_xi)
    self.w_hi.append(w_hi)
    self.b_i.append(b_i)

    return w_xi@input_x + w_hi@input_h + b_i

  def input_gate_g(self, input_x, input_h, output):
    w_xg = np.random.rand(output.shape[0], input_x.shape[0])
    w_hg = np.random.rand(output.shape[0], output.shape[1])
    b_g = np.random.rand(output.shape[0],1)
    
    self.w_xg.append(w_xg)
    self.w_hg.append(w_hg)
    self.b_g.append(b_g)

    return w_xg@input_x + w_hg@input_h + b_g

  def input_gate(self, input_x, input_h, output):
    lstm = LSTM()
    activation = Activation()

    i_t = lstm.input_gate_i(input_x, input_h, output)
    g_t = lstm.input_gate_g(input_x, input_h, output)

    activation_i = 2 * activation.sigmoid_fun(i_t) - 1
    activation_g = activation.sigmoid_fun(g_t)
    
    return activation_i * activation_g

  def forget_gate(self, input_x, input_h, output):
    activation = Activation()

    w_xf = np.random.rand(output.shape[0], input_x.shape[0])
    w_hf = np.random.rand(output.shape[0], output.shape[1])
    b_f = np.random.rand(output.shape[0],1)
    
    self.w_xf.append(w_xf)
    self.w_hf.append(w_hf)
    self.b_f.append(b_f)

    hidden_input = w_xf@input_x + w_hf@input_h + b_f
    
    return activation.sigmoid_fun(hidden_input)

  def output_gate(self, input_x, input_h, output):
    activation = Activation()

    w_xo = np.random.rand(output.shape[0], input_x.shape[0])
    w_ho = np.random.rand(output.shape[0], output.shape[1])
    b_o = np.random.rand(output.shape[0],1)
    
    self.w_xo.append(w_xo)
    self.w_ho.append(w_ho)
    self.b_o.append(b_o)

    hidden_input = w_xo@input_x + w_ho@input_x + b_o
    
    return activation.sigmoid_fun(hidden_input)



In [5]:
input = perch_length[:3].reshape(-1,1)

In [6]:
target = perch_length[:4].reshape(-1,1)

In [7]:
output = np.zeros((4,1))